# Yelp Businesses: Cleaning and Wrangling

The objective of this notebook is to inspect and wrangle the `business.json` file from the yelp dataset.
At each feature extracting/cleaning step, the data is saved in a separate csv file in the format `business_feature.csv` such that we can trace back the file origin. This is also to avoid ending up with a massing dataframe with too many features.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from collections import Counter, OrderedDict
import calendar
import sys
import os
sys.path.append(os.path.abspath(r"C:\ThacSi\HoachDinhCNTT\HDCNTT"))  # Thêm đường dẫn cha của thư mục wrangle
import utils

%matplotlib inline

# Load + Assess

In [6]:
#location of file
business_dir = '../data/business.json'

#download data
df_bus = utils.chunk_loader(business_dir, read_limit=-1)
#df_bus = pd.read_json(business_dir, orient='columns',lines=True)

#head
df_bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [7]:
#shape
df_bus.shape

(150346, 14)

In [8]:
#data types
df_bus.dtypes

business_id      object
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
stars           float64
review_count      int64
is_open           int64
attributes       object
categories       object
hours            object
dtype: object

In [9]:
#quick stats
df_bus.describe()

,latitude,longitude,stars,review_count,is_open
count,150346.000000,150346.000000,150346.000000,150346.000000,150346.00000
mean,36.671150,-89.357339,3.596724,44.866561,0.79615
std,5.872759,14.918502,0.974421,121.120136,0.40286
min,27.555127,-120.095137,1.000000,5.000000,0.00000
25%,32.187293,-90.357810,3.000000,8.000000,1.00000
50%,38.777413,-86.121179,3.500000,15.000000,1.00000
75%,39.954036,-75.421542,4.500000,37.000000,1.00000
max,53.679197,-73.200457,5.000000,7568.000000,1.00000


In [10]:
#percent missing
df_bus.isna().mean()

business_id     0.000000
name            0.000000
address         0.000000
city            0.000000
state           0.000000
postal_code     0.000000
latitude        0.000000
longitude       0.000000
stars           0.000000
review_count    0.000000
is_open         0.000000
attributes      0.091416
categories      0.000685
hours           0.154464
dtype: float64

# Cleaning to-do list

Evaluating the business dataframe above, there are several features that need to be cleaned.
The list below offers a roadmap to addressing these issues although it might not be comprehensive. During the process we might need to add additional steps.
We understand that some data types are nested within the columns, and that these data types might not be stored  in the appropirate manner.


- address
    - make everything lower case
    - extract feature: if on road/boulevard/ave/etc...
- attributes
    - break up dict to dummy variables
- business_id
    - no changes
- categories
    - make everything lower?
    - dummy variables and split by comma character
    - note that not everything is a restaurant (plumbers)
- city
    - maybe lower case?
- hours
    - split dict by days
        - open hour monday
        - close hour monday
        - etc...
    - figure out placeholder value for None
    - check if correlation between closed restaurant and no hours posted
- is_open
    - no changes
    - 82% are open, 18% are dead businesses
- latitude
    - no changes
- longitude
    - no changes
- name
    - no changes
- postal_code
    - replace zip code with integer representation
- review_count
    - note that lowest value is 3
- stars
    - no changes
- state
    - some are canadian
    - add feature: is in USA yes/no

In [11]:
#mapping dict for replacing and fixing data types
bool_to_int = {True: 1, False: 0, np.nan: 0, 'True': 1, 'False': 0, 'None': 0, None: 0}

## address

In [12]:
#create deep copy
df_bus_adr = df_bus.copy()
#make everything lower case
df_bus_adr['address'] = df_bus_adr['address'].str.lower()
#remove punctuation
df_bus_adr['address'] = df_bus_adr['address'].str.replace('[^\w\s]','')

In [13]:
#define counter object
adr_counter = Counter()
#loop over every address entry
for add in df_bus_adr.address:
    #loop over each individial word
    for word in add.lower().split():
        #add word to counter
        adr_counter[word] +=1

In [14]:
#list top k words
adr_counter.most_common(100)

[('st', 31736),
 ('ave', 22812),
 ('rd', 21587),
 ('ste', 19766),
 ('n', 17748),
 ('e', 13880),
 ('s', 13616),
 ('w', 12274),
 ('blvd', 11586),
 ('dr', 6852),
 ('pike', 5867),
 ('rd,', 4873),
 ('hwy', 4661),
 ('st,', 4339),
 ('main', 3028),
 ('nw', 2857),
 ('ave,', 2809),
 ('blvd,', 2674),
 ('state', 2633),
 ('street', 2507),
 ('us', 2354),
 ('100', 1810),
 ('avenue', 1797),
 ('dr,', 1784),
 ('ln', 1777),
 ('19', 1476),
 ('b', 1376),
 ('way', 1374),
 ('1', 1334),
 ('101', 1288),
 ('broadway', 1278),
 ('pkwy', 1247),
 ('a', 1225),
 ('lancaster', 1214),
 ('park', 1193),
 ('old', 1177),
 ('unit', 1169),
 ('dale', 1160),
 ('mabry', 1131),
 ('virginia', 1123),
 ('4th', 1087),
 ('west', 1077),
 ('market', 1062),
 ('road', 1024),
 ('broad', 987),
 ('2nd', 979),
 ('south', 973),
 ('pike,', 952),
 ('center', 940),
 ('washington', 905),
 ('200', 896),
 ('oracle', 869),
 ('manchester', 821),
 ('chestnut', 791),
 ('gulf', 783),
 ('east', 747),
 ('walnut', 746),
 ('ridge', 746),
 ('ln,', 741),
 ('b

In [15]:
#road type mapping to homogenize road names
road_type_dict = {'rd': 'road','rue': 'road', 'avenue': 'ave',
                  'street': 'str', 'blvd': 'boulevard',
                  'drive': 'dr', 'highway': 'hwy',
                  'parkway': 'pkwy', 'center': 'ct', 'lane': 'ln'}

#replace names
df_bus_adr['address'] = df_bus_adr['address'].replace(road_type_dict, regex=True)

In [16]:
#get list of finalized road values
#set to remove duplicates
road_types_list = list(set(road_type_dict.values()))
print(road_types_list)

['dr', 'ave', 'pkwy', 'str', 'ct', 'hwy', 'ln', 'road', 'boulevard']


In [17]:
#dict for dummies
road_col_dict = {}
#iterate over road types
for road in road_types_list:
    #create a dummy for that type
    dum_col = df_bus_adr['address'].str.contains(road)
    #add it to the dict
    road_col_dict[road] = dum_col

#convert boolean to 1/0
road_type_df = pd.DataFrame.from_dict(road_col_dict).replace({False:0, True:1})

road_type_df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15224\594145455.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  road_type_df = pd.DataFrame.from_dict(road_col_dict).replace({False:0, True:1})


,dr,ave,pkwy,str,ct,hwy,ln,road,boulevard
0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0


In [18]:
#key for reference in case we want to join tables
road_type_df['business_id'] = df_bus['business_id']

In [19]:
import os

# tạo thư mục nếu chưa có
os.makedirs('data/cleaned', exist_ok=True)

# lưu file
road_type_df.to_csv('data/cleaned/business_roadtype.csv', index=False)


### Alternatively we can create a categorical series instead of one hot encoding to reduce the dimensionality of the combined dataframe. We can do this because the categories are mutually exclusive.

In [20]:
#drop index col since we use dataframe later down
road_type_df = road_type_df.drop(columns=['business_id'])

In [21]:
#create dict mapping for road types
road_to_cat_dict = dict(zip(road_type_df.columns, np.arange(1, len(road_type_df.columns)+1)))

#stak dataframe
road_type_df_stack = road_type_df.stack()

#convert to series
road_type_df_cats = pd.Series(pd.Categorical(road_type_df_stack[road_type_df_stack!=0].index.get_level_values(1)))

#replace with dict
road_type_df_cats = road_type_df_cats.replace(road_to_cat_dict)


road_type_df_cats.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15224\309533305.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  road_type_df_cats = road_type_df_cats.replace(road_to_cat_dict)
C:\Users\Admin\AppData\Local\Temp\ipykernel_15224\309533305.py:11: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  road_type_df_cats = road_type_df_cats.replace(road_to_cat_dict)


0    5
1    8
2    9
3    7
4    8
dtype: category
Categories (9, int64): [2, 9, 5, 1, ..., 7, 3, 8, 4]

In [22]:
road_type_df_cats.shape

(102755,)

In [23]:
df_bus['road_type'] = road_type_df_cats

#fill missing values with 0
df_bus['road_type'] = df_bus['road_type'].astype('object').fillna(0)


C:\Users\Admin\AppData\Local\Temp\ipykernel_15224\1393366029.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_bus['road_type'] = df_bus['road_type'].astype('object').fillna(0)


### We can't necessarily rank the importance of the address based on its designation. A road might be more visited than a parkway, and the numbering of the road might not indicate anything. Thinking ahead, we believe that using one hot encoding might be the best solution for this categorical variable

### Replace postal codes with corresponding integer code

### Alternatively:
use IRS data to get income by zip code

https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi

https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi


For Canada:
https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/inc-rev/Table.cfm?Lang=Eng&T=102&PR=0&D1=1&RPP=25&SR=1&S=108&O=D



In [24]:
#get unique postal codes
unique_postal = df_bus['postal_code'].unique().tolist()

#map code to integer representation
postal_to_int = dict(zip(unique_postal, np.arange(0, len(unique_postal))))

#reverse mapping for reference
int_to_postal = {v:k for k,v in postal_to_int.items()}

#replace in dataframe
df_bus['postal_code'] = df_bus['postal_code'].replace(postal_to_int)

C:\Users\Admin\AppData\Local\Temp\ipykernel_15224\1211755027.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_bus['postal_code'] = df_bus['postal_code'].replace(postal_to_int)


## attributes

In [25]:
#break up dict inside df
df_atr = df_bus['attributes'].apply(pd.Series)
df_atr.head(10)

,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,BusinessParking,...,AcceptsInsurance,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions
0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,True,True,2,False,False,False,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,False,True,1,NaN,True,False,True,u'free',"{'garage': False, 'street': True, 'validated':...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,True,True,NaN,NaN,True,NaN,False,NaN,"{'garage': None, 'street': None, 'validated': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,False,True,False,1,False,True,True,False,u'no',None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,True,True,2,NaN,NaN,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,True,NaN,1,NaN,True,False,True,u'free',"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,False,True,NaN,1,False,True,True,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#inspect types
df_atr.dtypes

ByAppointmentOnly             object
BusinessAcceptsCreditCards    object
BikeParking                   object
RestaurantsPriceRange2        object
CoatCheck                     object
RestaurantsTakeOut            object
RestaurantsDelivery           object
Caters                        object
WiFi                          object
BusinessParking               object
WheelchairAccessible          object
HappyHour                     object
OutdoorSeating                object
HasTV                         object
RestaurantsReservations       object
DogsAllowed                   object
Alcohol                       object
GoodForKids                   object
RestaurantsAttire             object
Ambience                      object
RestaurantsTableService       object
RestaurantsGoodForGroups      object
DriveThru                     object
NoiseLevel                    object
GoodForMeal                   object
BusinessAcceptsBitcoin        object
Smoking                       object
M

In [27]:
#select cols that are still as objects
df_atr_obj_cols = df_atr.select_dtypes(include='object').columns
print(df_atr_obj_cols)

Index(['ByAppointmentOnly', 'BusinessAcceptsCreditCards', 'BikeParking',
       'RestaurantsPriceRange2', 'CoatCheck', 'RestaurantsTakeOut',
       'RestaurantsDelivery', 'Caters', 'WiFi', 'BusinessParking',
       'WheelchairAccessible', 'HappyHour', 'OutdoorSeating', 'HasTV',
       'RestaurantsReservations', 'DogsAllowed', 'Alcohol', 'GoodForKids',
       'RestaurantsAttire', 'Ambience', 'RestaurantsTableService',
       'RestaurantsGoodForGroups', 'DriveThru', 'NoiseLevel', 'GoodForMeal',
       'BusinessAcceptsBitcoin', 'Smoking', 'Music', 'GoodForDancing',
       'AcceptsInsurance', 'BestNights', 'BYOB', 'Corkage', 'BYOBCorkage',
       'HairSpecializesIn', 'Open24Hours', 'RestaurantsCounterService',
       'AgesAllowed', 'DietaryRestrictions'],
      dtype='object')


In [28]:
df_atr[df_atr_obj_cols].head(10)

,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,BusinessParking,...,AcceptsInsurance,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions
0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,True,True,2,False,False,False,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,False,True,1,NaN,True,False,True,u'free',"{'garage': False, 'street': True, 'validated':...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,True,True,NaN,NaN,True,NaN,False,NaN,"{'garage': None, 'street': None, 'validated': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,False,True,False,1,False,True,True,False,u'no',None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,True,True,2,NaN,NaN,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,True,NaN,1,NaN,True,False,True,u'free',"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,False,True,NaN,1,False,True,True,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
def str_dict_to_df(series):
    """
    Takes in a pandas series with dicts stored as strings
    returns dataframe with dict keys as columns
    
    serires: pandas series
    """
    eval_list = []
    for sr in series:
        if not pd.isna(sr):
            eval_list.append(eval(sr))
        else:
            eval_list.append(np.nan)
    
    eval_df = pd.Series(eval_list).apply(pd.Series)  
    
    #drop cols that are all nan
    eval_df = eval_df.dropna(axis=1, how='all')
    
    
    return eval_df

In [31]:
#store col names
dict_cols_list = []

for col in df_atr_obj_cols:
    #if contains a curly bracket, then assume column is a dict as string
    if df_atr[col].str.contains('{').any():
        dict_cols_list.append(col)
print(dict_cols_list)            

['BusinessParking', 'Ambience', 'GoodForMeal', 'Music', 'BestNights', 'HairSpecializesIn', 'DietaryRestrictions']


In [32]:
#store dataframes from dict nested columns
dict_col_df_list = []

for col in dict_cols_list:
    #apply string to dict evaluation
    temp_df = str_dict_to_df(df_atr[col])
    #append to list
    dict_col_df_list.append(temp_df)

#combine all in one column
dict_col_df = pd.concat(dict_col_df_list, axis=1)
dict_col_df.head()

,garage,street,validated,lot,valet,romantic,intimate,touristy,hipster,divey,...,kids,perms,asian,dairy-free,gluten-free,vegan,kosher,halal,soy-free,vegetarian
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,False,False,True,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,False,True,False,False,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,None,None,True,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
#store dummy attribute columns in list
atr_dum_df_cols = []

#iterate over object columns
for col in df_atr_obj_cols:
    #if contains a categorical variable then it begins with a u
    if df_atr[col].str.contains("u'").any():
        #add to the list
        atr_dum_df_cols.append(col)

#print out list
print(atr_dum_df_cols)

['WiFi', 'BusinessParking', 'Alcohol', 'RestaurantsAttire', 'Ambience', 'NoiseLevel', 'GoodForMeal', 'Smoking', 'Music', 'BestNights', 'BYOBCorkage', 'HairSpecializesIn', 'AgesAllowed']


In [34]:
#explore unique entries in attribute columns
for col in atr_dum_df_cols:
    print(col)
    print(df_atr[col].unique())
    print(50*'-')


WiFi
[nan "u'no'" "u'free'" "'free'" "'no'" 'None' "u'paid'" "'paid'"]
--------------------------------------------------
BusinessParking
[nan
 "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"
 "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"
 "{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}"
 'None'
 "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"
 "{'garage': None, 'street': False, 'validated': None, 'lot': True, 'valet': False}"
 "{u'valet': False, u'garage': None, u'street': True, u'lot': False, u'validated': None}"
 "{'garage': False, 'street': True, 'validated': False, 'lot': True, 'valet': False}"
 "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}"
 "{'garage': True, 'street': False, 'validated': True, 'lot': False, 'valet': True}"
 "{'garage': None, 'street': True, 'validated': None, 'lot': False, 'v

### We build a dictionary to replace each categorical variable with a numerical value that corresponds to a sliding scale. For example we can rank attire from casual, to dressy, and to formal. This will reduce the feature space, and probably provide more information gain than a one hot encoding. For simplicity we will assume that a NaN is always the lowest value on the scale.

In [35]:
#noise level
noise_level_dict = dict(zip([np.nan, "u'loud'" ,"u'average'" ,
                             "'average'" ,"u'quiet'" ,"'loud'" ,
                             "'very_loud'", "'quiet'" ,"u'very_loud'" ,'None'], 
                            [0, 3, 2, 
                             2, 1, 3, 
                             4, 1 , 4, 0]))
print(noise_level_dict)




{nan: 0, "u'loud'": 3, "u'average'": 2, "'average'": 2, "u'quiet'": 1, "'loud'": 3, "'very_loud'": 4, "'quiet'": 1, "u'very_loud'": 4, 'None': 0}


In [36]:
#free wifi is the best kind of wifi
wifi_dict = dict(zip(
    [np.nan ,"u'no'" ,"'free'" ,
     "u'free'" ,"'no'" ,"u'paid'" ,"'paid'" ,'None'], 
    [0, 1, 3, 
     3, 1, 2, 2, 0]))
print(wifi_dict)

{nan: 0, "u'no'": 1, "'free'": 3, "u'free'": 3, "'no'": 1, "u'paid'": 2, "'paid'": 2, 'None': 0}


In [37]:
#more alcohol is better
alcohol_dict = dict(zip([np.nan ,"u'full_bar'" ,"u'beer_and_wine'" ,
                         "u'none'" ,"'beer_and_wine'" ,"'none'", 
                         "'full_bar'" ,'None'],
                        [0, 2, 1, 
                         0, 1, 0, 
                         2, 0]))
print(alcohol_dict)

{nan: 0, "u'full_bar'": 2, "u'beer_and_wine'": 1, "u'none'": 0, "'beer_and_wine'": 1, "'none'": 0, "'full_bar'": 2, 'None': 0}


In [38]:
#rank by formality
restaurant_attire_dict = dict(zip(
    [np.nan ,"u'casual'" ,"'casual'" ,
     "'dressy'" ,"u'dressy'" ,"u'formal'" ,
     'None',"'formal'"], 
    [0, 1, 1, 
     2, 2, 3, 
     0, 3]))
print(restaurant_attire_dict)

{nan: 0, "u'casual'": 1, "'casual'": 1, "'dressy'": 2, "u'dressy'": 2, "u'formal'": 3, 'None': 0, "'formal'": 3}


In [39]:
#free corkage is best
byob_corkage_dict = dict(zip(
    [np.nan ,"'no'" ,"'yes_corkage'" ,
     "'yes_free'" ,"u'no'" ,"u'yes_free'","u'yes_corkage'"],
    [0, 1, 2, 
     3, 1, 3, 2]))
print(byob_corkage_dict)

{nan: 0, "'no'": 1, "'yes_corkage'": 2, "'yes_free'": 3, "u'no'": 1, "u'yes_free'": 3, "u'yes_corkage'": 2}


In [40]:
#from least to most accomodating for smokers
smoking_dict = dict(zip(
    [np.nan ,"u'no'" ,"u'outdoor'" ,
     "u'yes'" ,"'no'" ,'None' ,
     "'outdoor'" ,"'yes'"], 
    [0, 1, 2, 
     3, 1, 0, 
     2, 3]))
print(smoking_dict)

{nan: 0, "u'no'": 1, "u'outdoor'": 2, "u'yes'": 3, "'no'": 1, 'None': 0, "'outdoor'": 2, "'yes'": 3}


In [41]:
#from youngest to oldest crowd
ages_allowed_dict = dict(zip(
    [np.nan ,"u'allages'" ,"u'21plus'" ,
     "u'18plus'" ,"u'19plus'" ,'None'], 
    [0, 1, 4, 
     2, 3, 0]))
print(ages_allowed_dict)

{nan: 0, "u'allages'": 1, "u'21plus'": 4, "u'18plus'": 2, "u'19plus'": 3, 'None': 0}


In [42]:
#define mapping for replace method
atr_dict_map = {'NoiseLevel': noise_level_dict, 
                'WiFi': wifi_dict, 
                'Alcohol': alcohol_dict, 
                'RestaurantsAttire': restaurant_attire_dict, 
                'BYOBCorkage': byob_corkage_dict, 
                'Smoking': smoking_dict, 
                'AgesAllowed': ages_allowed_dict}

In [43]:
#get a copy with relevant columns
df_atr_categorical = df_atr[atr_dum_df_cols].copy()
#apply replacement
df_atr_categorical = df_atr_categorical.replace(atr_dict_map)

df_atr_categorical.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15224\1507130580.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_atr_categorical = df_atr_categorical.replace(atr_dict_map)


,WiFi,BusinessParking,Alcohol,RestaurantsAttire,Ambience,NoiseLevel,GoodForMeal,Smoking,Music,BestNights,BYOBCorkage,HairSpecializesIn,AgesAllowed
0,0,NaN,0,0,NaN,0,NaN,0,NaN,NaN,0,NaN,0
1,0,NaN,0,0,NaN,0,NaN,0,NaN,NaN,0,NaN,0
2,1,"{'garage': False, 'street': False, 'validated'...",0,0,NaN,0,NaN,0,NaN,NaN,0,NaN,0
3,3,"{'garage': False, 'street': True, 'validated':...",0,0,NaN,0,NaN,0,NaN,NaN,0,NaN,0
4,0,"{'garage': None, 'street': None, 'validated': ...",0,0,NaN,0,NaN,0,NaN,NaN,0,NaN,0


#### The cell block below will return a one hot encoding of the categorical attributes above, uncomment if objective changes.

In [44]:
# #store dummy dataframes in list
# atr_dum_df_list = []

# for col in atr_dum_df_cols:
    
#     #we do not want to modify the dataframe in place so create a copy
#     temp_series = df_atr[col].copy()
    
    
#     #fx messy inputs and remove u
#     temp_series = temp_series.str.replace("u'", "")
#     #remove '
#     temp_series= temp_series.str.replace("'", "")
        
    
#     #create dummies
#     dum_df = pd.get_dummies(temp_series)
#     #drop the None column
#     dum_df = dum_df.drop(columns=['None'])
        
#     #fx messy col names in case we missed them
#     dum_df.columns = dum_df.columns.str.replace("u'", "")
#     dum_df.columns = dum_df.columns.str.replace("'", "")
        
#     #add prefixt
#     dum_df = dum_df.add_prefix(col+'_')
                
#     #append to list
#     atr_dum_df_list.append(dum_df)

# #concat
# atr_dum_df = pd.concat(atr_dum_df_list, axis=1)

# atr_dum_df.head()


In [40]:
# #standard preprocessing for restaurant price range since it has no u
# atr_price_range = pd.get_dummies(df_atr['RestaurantsPriceRange2'])
# atr_price_range = atr_price_range.drop(columns= ['None'])
# atr_price_range = atr_price_range.add_prefix('price_range'+'_')

# atr_price_range.head()

In [45]:
#add to the list
atr_to_drop =  dict_cols_list + list(df_atr_categorical.columns)
print(atr_to_drop)

['BusinessParking', 'Ambience', 'GoodForMeal', 'Music', 'BestNights', 'HairSpecializesIn', 'DietaryRestrictions', 'WiFi', 'BusinessParking', 'Alcohol', 'RestaurantsAttire', 'Ambience', 'NoiseLevel', 'GoodForMeal', 'Smoking', 'Music', 'BestNights', 'BYOBCorkage', 'HairSpecializesIn', 'AgesAllowed']


In [46]:
#combine features in one dataframes
df_atr_conc = pd.concat([df_atr.drop(columns=atr_to_drop),dict_col_df, df_atr_categorical], axis=1)

#standardize name
df_atr_conc.columns = df_atr_conc.columns.str.replace("-", "_")

#make 1/0
df_atr_conc = df_atr_conc.replace(bool_to_int)

df_atr_conc.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15224\2415880967.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_atr_conc = df_atr_conc.replace(bool_to_int)


,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WheelchairAccessible,HappyHour,...,RestaurantsAttire,Ambience,NoiseLevel,GoodForMeal,Smoking,Music,BestNights,BYOBCorkage,HairSpecializesIn,AgesAllowed
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,2,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
#quick check for column names
df_atr_conc.columns

Index(['ByAppointmentOnly', 'BusinessAcceptsCreditCards', 'BikeParking',
       'RestaurantsPriceRange2', 'CoatCheck', 'RestaurantsTakeOut',
       'RestaurantsDelivery', 'Caters', 'WheelchairAccessible', 'HappyHour',
       'OutdoorSeating', 'HasTV', 'RestaurantsReservations', 'DogsAllowed',
       'GoodForKids', 'RestaurantsTableService', 'RestaurantsGoodForGroups',
       'DriveThru', 'BusinessAcceptsBitcoin', 'GoodForDancing',
       'AcceptsInsurance', 'BYOB', 'Corkage', 'Open24Hours',
       'RestaurantsCounterService', 'garage', 'street', 'validated', 'lot',
       'valet', 'romantic', 'intimate', 'touristy', 'hipster', 'divey',
       'classy', 'trendy', 'upscale', 'casual', 'dessert', 'latenight',
       'lunch', 'dinner', 'brunch', 'breakfast', 'dj', 'background_music',
       'no_music', 'jukebox', 'live', 'video', 'karaoke', 'monday', 'tuesday',
       'friday', 'wednesday', 'thursday', 'sunday', 'saturday',
       'straightperms', 'coloring', 'extensions', 'africanamerican

In [48]:
#key for reference
df_atr_conc['business_id'] = df_bus['business_id']

In [49]:
#save the work
df_atr_conc.to_csv(path_or_buf='data/cleaned/business_attributes.csv')

## Categories

In [50]:
#counter for original categories
cat_counter = Counter()
#loop through split categories
for cat_split in df_bus['categories'].str.split(',| '):
    #if statement to avoid none type is not iterable
    if cat_split:
        for cat in cat_split:
            cat_counter[cat] +=1

In [51]:
#see top k types
cat_counter.most_common(25)

[('', 518349),
 ('&', 108359),
 ('Services', 56156),
 ('Restaurants', 52343),
 ('Food', 43813),
 ('Shopping', 24858),
 ('Home', 24169),
 ('Bars', 21022),
 ('Spas', 17070),
 ('Beauty', 15836),
 ('American', 15046),
 ('Medical', 14073),
 ('Event', 13773),
 ('Health', 12930),
 ('Local', 12842),
 ('Nightlife', 12281),
 ('Planning', 11757),
 ('Hair', 11698),
 ('Repair', 11456),
 ('Stores', 11381),
 ('Automotive', 10773),
 ('Salons', 10544),
 ('Auto', 9812),
 ('Hotels', 8834),
 ('Sandwiches', 8366)]

In [52]:
#get keys for top k common categories
top_cats = list(dict(cat_counter.most_common(25)).keys())
print(top_cats)
#note that the first 2 were space and & so skip those
print("\nselecting only top 10 relevant sections\n")
top_cats = top_cats[3:13]
print(top_cats)

['', '&', 'Services', 'Restaurants', 'Food', 'Shopping', 'Home', 'Bars', 'Spas', 'Beauty', 'American', 'Medical', 'Event', 'Health', 'Local', 'Nightlife', 'Planning', 'Hair', 'Repair', 'Stores', 'Automotive', 'Salons', 'Auto', 'Hotels', 'Sandwiches']

selecting only top 10 relevant sections

['Restaurants', 'Food', 'Shopping', 'Home', 'Bars', 'Spas', 'Beauty', 'American', 'Medical', 'Event']


In [53]:
#create dict for categories
category_dict = {}
for cat in top_cats:
    #select relevant top 10 categories from before
    dum_cat = df_bus['categories'].str.contains(cat)
    #add it to the dict
    category_dict[cat] = dum_cat
    
cat_type_df = pd.DataFrame.from_dict(category_dict).replace(bool_to_int)

cat_type_df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_15224\968302208.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cat_type_df = pd.DataFrame.from_dict(category_dict).replace(bool_to_int)


,Restaurants,Food,Shopping,Home,Bars,Spas,Beauty,American,Medical,Event
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
#key for reference
cat_type_df['business_id'] = df_bus['business_id']

In [55]:
#save the work
cat_type_df.to_csv(path_or_buf='data/cleaned/business_cats.csv')

## Hours

Holy grail of date time format:

http://strftime.org/

In [56]:
#split out the dict
hours_day_df = df_bus['hours'].apply(pd.Series)
hours_day_df.head()

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,NaN
2,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
3,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,NaN,NaN,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0


In [57]:
def series_to_datetime(df, series):
    """
    Takes in a pandas series with content stored as a string
    must have format hour:minute - hour:minute
    can have missing values
    
    df = pandas dataframe
    series = pandas column name
    """
    #create array for day of week
    #weekday = list(calendar.day_abbr)
    
    #ordered dict container
    serires_dict = OrderedDict()
    
    #hour container
    open_hour = []
    close_hour = []
    
    #split the series along the dash (-)
    day = df[series].str.split("-")

    #iterate over days
    for hour in day:
        
        #if not a nan the split will return a list
        if type(hour)==list:
            open_hour.append(hour[0])
            close_hour.append(hour[1])
        else:
            #necessary nan for when not available
            open_hour.append(np.nan)
            close_hour.append(np.nan)
            
    #make a datetime object    
    
    open_hour_dt = pd.to_datetime(open_hour, dayfirst=True,format='%H:%M')
    close_hour_dt = pd.to_datetime(close_hour, dayfirst=True, format='%H:%M')
    
    serires_dict[series+'_open'] = open_hour
    serires_dict[series+'_close'] = close_hour
    
    hours_df = pd.DataFrame.from_dict(serires_dict)
    
    return hours_df

In [58]:
hours_df_list = []

for col in hours_day_df.columns:
    temp_hour_df = series_to_datetime(hours_day_df, col)
    hours_df_list.append(temp_hour_df)
    
    
hours_df_openclose = pd.concat(hours_df_list, axis=1)
hours_df_openclose.head()

,Monday_open,Monday_close,Tuesday_open,Tuesday_close,Wednesday_open,Wednesday_close,Thursday_open,Thursday_close,Friday_open,Friday_close,Saturday_open,Saturday_close,Sunday_open,Sunday_close
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0:0,0:0,8:0,18:30,8:0,18:30,8:0,18:30,8:0,18:30,8:0,14:0,NaN,NaN
2,8:0,22:0,8:0,22:0,8:0,22:0,8:0,22:0,8:0,23:0,8:0,23:0,8:0,22:0
3,7:0,20:0,7:0,20:0,7:0,20:0,7:0,20:0,7:0,21:0,7:0,21:0,7:0,21:0
4,NaN,NaN,NaN,NaN,14:0,22:0,16:0,22:0,12:0,22:0,12:0,22:0,12:0,18:0


In [59]:
hours_df_openclose.dtypes

Monday_open        object
Monday_close       object
Tuesday_open       object
Tuesday_close      object
Wednesday_open     object
Wednesday_close    object
Thursday_open      object
Thursday_close     object
Friday_open        object
Friday_close       object
Saturday_open      object
Saturday_close     object
Sunday_open        object
Sunday_close       object
dtype: object

In [60]:
#key for reference
hours_df_openclose['business_id'] = df_bus['business_id']

In [61]:
#save the work
hours_df_openclose.to_csv(path_or_buf='data/cleaned/business_hours.csv')

## is_open

In [62]:
#quick check for unique values
df_bus.is_open.unique()

array([0, 1])

## state

In [63]:
#USA! USA!

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

In [64]:
#get usa sates and make dataframe
df_is_usa = pd.DataFrame(df_bus['state'].str.contains('|'.join(states)).astype(int))
df_is_usa['business_id'] = df_bus['business_id']
#drop redundant column
df_is_usa = df_is_usa.drop(columns=['state'])

In [65]:
#save the work
df_is_usa.to_csv(path_or_buf='data/cleaned/is_usa.csv', header=True)

### Combine DFs in master DF with business_id as common key

In [ ]:
# #superseded in favor for utils.reduce_merge

# #combine in massive df
# df_list = [df_bus, 
#            road_type_df_cats, 
#            df_atr_conc.drop(columns=['business_id']), 
#            cat_type_df.drop(columns=['business_id']), 
#            hours_df_openclose.drop(columns=['business_id']), 
#            df_is_usa.drop(columns=['business_id'])]

# df_bus_conc = pd.concat(df_list, axis=1)

# df_bus_conc.shape

In [67]:
#combine in massive df
df_list = [df_bus, 
           df_atr_conc, 
           cat_type_df, 
           hours_df_openclose, 
           df_is_usa]

df_bus_conc = utils.reduce_merge(df_list, key='business_id')

df_bus_conc.shape

(150346, 126)

In [68]:
#check columns
print(*df_bus_conc.columns, sep=',   ')

business_id,   name,   address,   city,   state,   postal_code,   latitude,   longitude,   stars,   review_count,   is_open,   attributes,   categories,   hours,   road_type,   ByAppointmentOnly,   BusinessAcceptsCreditCards,   BikeParking,   RestaurantsPriceRange2,   CoatCheck,   RestaurantsTakeOut,   RestaurantsDelivery,   Caters,   WheelchairAccessible,   HappyHour,   OutdoorSeating,   HasTV,   RestaurantsReservations,   DogsAllowed,   GoodForKids,   RestaurantsTableService,   RestaurantsGoodForGroups,   DriveThru,   BusinessAcceptsBitcoin,   GoodForDancing,   AcceptsInsurance,   BYOB,   Corkage,   Open24Hours,   RestaurantsCounterService,   garage,   street,   validated,   lot,   valet,   romantic,   intimate,   touristy,   hipster,   divey,   classy,   trendy,   upscale,   casual,   dessert,   latenight,   lunch,   dinner,   brunch,   breakfast,   dj,   background_music,   no_music,   jukebox,   live,   video,   karaoke,   monday,   tuesday,   friday,   wednesday,   thursday,   su

In [69]:
#save the work
df_bus_conc.to_csv(path_or_buf='data/cleaned/df_bus_conc.csv', header=True)


In [72]:
import os
os.makedirs('data/cleaned', exist_ok=True)

print("Saving file...")
df_bus.to_csv('data/cleaned/business_merge_density_added.csv', index=False)

print("✅ Done. File saved at: data/cleaned/business_merge_density_added.csv")


Saving file...
✅ Done. File saved at: data/cleaned/business_merge_density_added.csv


In [73]:
df_bus.to_csv('data/cleaned/business_merge_density_added.csv', index=False)
